In [1]:
from jetbot import Robot
robot = Robot()
robot.stop()

In [2]:
%run DEBUGGING_Angle.ipynb
%run DEBUGGING_Lidar.ipynb
%run Scripts/Env_Class.ipynb

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-5_r5rike because the default path (/home/jetbot/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Matplotlib is building the font cache; this may take a moment.


KeyPoints RCNN Successfully Loaded. 


# Variables

In [3]:
# Define what directions you want LiDAR to measure
angles = [180, 200, 220, 240, 260, 280, 300, 320, 340, 0, 20, 40, 60, 80, 100, 120, 140, 160]

# Initialize all sensor readings to 0
previous_readings = {x:0 for x in angles} 

# Define Model
UNITY_MODEL = "Files/robot.pth"
model = torch.load(UNITY_MODEL)
model.eval()

SeparateActorCritic(
  (network_body): NetworkBody(
    (visual_processors): ModuleList()
    (vector_processors): ModuleList(
      (0): VectorInput(
        (normalizer): Normalizer()
      )
    )
    (linear_encoder): LinearEncoder(
      (seq_layers): Sequential(
        (0): Linear(in_features=22, out_features=512, bias=True)
        (1): Swish()
        (2): Linear(in_features=512, out_features=512, bias=True)
        (3): Swish()
      )
    )
  )
  (distribution): GaussianDistribution(
    (mu): Linear(in_features=512, out_features=2, bias=True)
  )
  (critic): ValueNetwork(
    (network_body): NetworkBody(
      (visual_processors): ModuleList()
      (vector_processors): ModuleList(
        (0): VectorInput(
          (normalizer): Normalizer()
        )
      )
      (linear_encoder): LinearEncoder(
        (seq_layers): Sequential(
          (0): Linear(in_features=22, out_features=512, bias=True)
          (1): Swish()
          (2): Linear(in_features=512, out_features=5

# Thread to control car & print information

In [4]:
def print_angle_thread():
    
    global print_angle_running
    global global_angle
    print_angle_running = True
    global angles
    global previous_readings
    
    while True:
        
        if len(keypoints.values()) > 1:
            parts = ["left_shoulder", "right_shoulder"]
#             parts = ["left_ankle", "right_ankle"]
            empty = [x for x in parts if x in keypoints.keys()]
            
            #stage 1 
            if len(empty) > 1: 
                stage = [1,0,0]
                angle, _ = calculate_angle(WIDTH, keypoints, FOV)
                
                # If we already assigned value to global angle
                if global_angle:
                    
                    # If the difference in angle is too large, update
                    if abs(global_angle-angle[0])>4:
                        global_angle = angle[0]
                        
                    # The angle is large enough to update
                    else:
                        global_angle = angle[0]
                
                # If first time running the while loop, assign angle to global variable
                else:
                    global_angle = angle[0]
                    
                        
            else: #stage 2
                stage = [0,1,0]
                global_angle = 0
                
        else: #stage 3
            stage = [0,0,1]
            global_angle = 0
            
        
        previous_readings = read_lidar_wrapper(angles, previous_readings)   # Read Lidar
            
        # Printing Information       
        observation = stage + [global_angle] + list(previous_readings.values())                # Join stage, angle and lidar readings
        observation2 = observation[0:4] + [x if x < 4000 else 4000 for x in observation[4:]]   # Clip lidar readings to 4m
        observation2 = observation2[0:4] + [round(x/4000,3) for x in observation2[4:]]         # Normalize lidar readings
        
        action, reward = env.sample_action(observation2)
        
        
        if stage == [1,0,0] and abs(global_angle)<=40:
            action[0] = 0
            action[1] = 0
            
        elif stage == [1,0,0] and abs(global_angle)>40:
            action[0] = 0
            
        action_move, action_turn = action.cpu().detach().numpy()
       
        env.step(action)

        lidar_info_print.value = f"<pref><font color='black'><span style='background-color:#A9CDF8'>*    Observation    *</span><font color='blue'><br>Stage: {observation2[0:3]}<br>Angle: {observation2[3]}<br>Lidar 0°: {observation2[13]}<br>Lidar 20°: {observation2[14]}<br>Lidar 40°: {observation2[15]}<br>Lidar 60°: {observation2[16]}<br>Lidar 80°: {observation2[17]}<br>Lidar 100°: {observation2[18]}<br>Lidar 120°: {observation2[19]}<br>Lidar 140°: {observation2[20]}<br>Lidar 160°: {observation2[21]}<br>Lidar 180°: {observation2[4]}<br>Lidar 200°: {observation2[5]}<br>Lidar 220°: {observation2[6]}<br>Lidar 240°: {observation2[7]}<br>Lidar 260°: {observation2[8]}<br>Lidar 280°: {observation2[9]}<br>Lidar 300°: {observation2[10]}<br>Lidar 320°: {observation2[11]}<br>Lidar 340°: {observation2[12]}</pref>"
        
        action_reward_info.value = f"<pref><font color='black'><span style='background-color:#E1A3A3'>*    Action Taken   *</span><font color='red'><br>Move: {round(float(action_move),3)}<br>Rotate: {round(float(action_turn),3)}<br>\
                                           <font color='black'><span style='background-color:#CFBBE8'>*       Reward      *</span><font color='purple'><br>{round(float(reward),3)}</pref>"


# Load Model and prepare environment

In [5]:
env = Environment(model, angles, robot)

# Control threads

In [6]:
''' Image Widget'''
# Image 
image_w = ipywidgets.Image(format='jpeg', width=500, height=HEIGHT)

global lidar_info_print
lidar_info_print = widgets.HTML()

global action_reward_info
action_reward_info = widgets.HTML()

''' Join all widgets into a single HBox'''
# Visualize the whole thing (sliders + buttons)
final = widgets.HBox([lidar_info_print, image_w, action_reward_info], layout=widgets.Layout(width='100%', align_self='center'))
display(final)

execute({'new': camera.value})
camera.observe(execute, names='value')